In [ ]:
# Polynomial Regression: Complete Google Colab Example
# This example demonstrates polynomial regression using a synthetic dataset
# that exhibits a clear non-linear relationship

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("=== Polynomial Regression Demonstration ===\n")

# Step 1: Generate synthetic dataset with non-linear relationship
print("Step 1: Generating synthetic dataset...")

# Create a dataset representing "Study Hours vs Test Score" with diminishing returns
n_samples = 100
X = np.linspace(0, 10, n_samples).reshape(-1, 1)  # Study hours (0-10)

# True relationship: quadratic with some noise
# Score = 20 + 15*hours - 0.8*hours^2 + noise
# This creates an inverted U-shape (diminishing returns)
true_function = 20 + 15*X.ravel() - 0.8*X.ravel()**2
noise = np.random.normal(0, 3, n_samples)  # Add realistic noise
y = true_function + noise

# Ensure scores are realistic (0-100 range)
y = np.clip(y, 0, 100)

print(f"Dataset created: {n_samples} samples")
print(f"X range: {X.min():.1f} to {X.max():.1f} hours")
print(f"y range: {y.min():.1f} to {y.max():.1f} points\n")

# Step 2: Create and compare different polynomial degrees
print("Step 2: Training polynomial regression models...")

degrees = [1, 2, 3, 4]  # Test different polynomial degrees
models = {}
predictions = {}
r2_scores = {}

# Train models for each degree
for degree in degrees:
    print(f"Training degree {degree} polynomial...")

    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)

    # Create pipeline: Polynomial Features -> Linear Regression
    model = Pipeline([
        ('poly_features', poly_features),
        ('linear_regression', LinearRegression())
    ])

    # Train the model
    model.fit(X, y)

    # Make predictions
    y_pred = model.predict(X)

    # Calculate R-squared score
    r2 = r2_score(y, y_pred)

    # Store results
    models[degree] = model
    predictions[degree] = y_pred
    r2_scores[degree] = r2

    print(f"  R-squared score: {r2:.4f}")

print("\n" + "="*50)

# Step 3: Display model performance comparison
print("Step 3: Model Performance Comparison")
print("-" * 40)
for degree in degrees:
    mse = mean_squared_error(y, predictions[degree])
    print(f"Degree {degree}: R² = {r2_scores[degree]:.4f}, MSE = {mse:.2f}")

# Find best model based on R-squared
best_degree = max(r2_scores, key=r2_scores.get)
print(f"\nBest performing model: Degree {best_degree} (R² = {r2_scores[best_degree]:.4f})")

print("\n" + "="*50)

# Step 4: Detailed analysis of the best model
print("Step 4: Detailed Analysis of Best Model")
print("-" * 40)

best_model = models[best_degree]
best_poly_features = best_model.named_steps['poly_features']
best_linear_reg = best_model.named_steps['linear_regression']

# Get feature names and coefficients
feature_names = best_poly_features.get_feature_names_out(['hours'])
coefficients = best_linear_reg.coef_
intercept = best_linear_reg.intercept_

print(f"Polynomial Equation (degree {best_degree}):")
print(f"Test Score = {intercept:.2f}", end="")
for i, (name, coef) in enumerate(zip(feature_names, coefficients)):
    sign = "+" if coef >= 0 else "-"
    print(f" {sign} {abs(coef):.2f}*{name}", end="")
print("\n")

# Step 5: Create comprehensive visualization
print("Step 5: Creating visualization...")

# Create figure with subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Polynomial Regression Analysis: Study Hours vs Test Scores', fontsize=16, fontweight='bold')

# Colors for different degrees
colors = ['red', 'green', 'blue', 'orange']

# Plot 1: All polynomial fits
ax1.scatter(X, y, alpha=0.6, color='black', s=30, label='Data points')
for i, degree in enumerate(degrees):
    ax1.plot(X, predictions[degree], color=colors[i], linewidth=2,
             label=f'Degree {degree} (R²={r2_scores[degree]:.3f})')
ax1.set_xlabel('Study Hours')
ax1.set_ylabel('Test Score')
ax1.set_title('Comparison of Different Polynomial Degrees')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Focus on best model
ax2.scatter(X, y, alpha=0.6, color='black', s=30, label='Actual data')
ax2.plot(X, predictions[best_degree], color='red', linewidth=3,
         label=f'Best fit (Degree {best_degree})')
ax2.set_xlabel('Study Hours')
ax2.set_ylabel('Test Score')
ax2.set_title(f'Best Model: Degree {best_degree} Polynomial')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Residuals analysis
residuals = y - predictions[best_degree]
ax3.scatter(predictions[best_degree], residuals, alpha=0.6, color='blue', s=30)
ax3.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax3.set_xlabel('Predicted Test Score')
ax3.set_ylabel('Residuals')
ax3.set_title('Residuals Plot (Best Model)')
ax3.grid(True, alpha=0.3)

# Plot 4: R-squared comparison
ax4.bar(range(len(degrees)), [r2_scores[d] for d in degrees],
        color=colors, alpha=0.7)
ax4.set_xlabel('Polynomial Degree')
ax4.set_ylabel('R-squared Score')
ax4.set_title('Model Performance Comparison')
ax4.set_xticks(range(len(degrees)))
ax4.set_xticklabels([f'Degree {d}' for d in degrees])
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, degree in enumerate(degrees):
    ax4.text(i, r2_scores[degree] + 0.01, f'{r2_scores[degree]:.3f}',
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*50)

# Step 6: Making predictions with the best model
print("Step 6: Making Predictions")
print("-" * 30)

# Example predictions for new data points
new_hours = np.array([[2], [5], [8]])
new_predictions = best_model.predict(new_hours)

print("Predictions for new study hours:")
for hrs, pred in zip(new_hours.flatten(), new_predictions):
    print(f"  {hrs} hours/week → {pred:.1f} points")

print("\n" + "="*50)

# Step 7: Key insights and interpretation
print("Step 7: Key Insights")
print("-" * 20)

print("1. RELATIONSHIP PATTERN:")
if best_degree == 2:
    print("   • Quadratic relationship detected (inverted U-shape)")
    print("   • Shows diminishing returns pattern")
    print("   • Optimal study time exists where performance peaks")
elif best_degree == 3:
    print("   • Cubic relationship detected (S-shape)")
    print("   • More complex pattern with multiple inflection points")
else:
    print(f"   • Degree {best_degree} polynomial provides best fit")

print("\n2. PRACTICAL IMPLICATIONS:")
print("   • Initial study hours show strong positive returns")
print("   • Additional hours beyond optimal point show diminishing returns")
print("   • Over-studying might lead to decreased performance")

print("\n3. MODEL QUALITY:")
print(f"   • Best model explains {r2_scores[best_degree]*100:.1f}% of variance")
print(f"   • Residuals appear randomly distributed (good sign)")

# Find optimal study hours (if quadratic)
if best_degree == 2:
    optimal_hours = -coefficients[0] / (2 * coefficients[1])
    if 0 <= optimal_hours <= 10:  # Within our data range
        optimal_score = best_model.predict([[optimal_hours]])[0]
        print(f"   • Optimal study time: ~{optimal_hours:.1f} hours (score: {optimal_score:.1f})")

print("\n" + "="*50)
print("Analysis Complete! 🎉")
print("\nKey Takeaways:")
print("• Polynomial regression successfully captured non-linear relationship")
print("• Higher degrees don't always mean better models (risk of overfitting)")
print("• Visual inspection and residual analysis are crucial for model validation")
print("• The model provides actionable insights about optimal study strategies")